This is a step by step guide for web scraping using BeautifulSoup

In [26]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from http.client import RemoteDisconnected

In [27]:
categories_path = [
    "Immobilier", 
    "ImmoNeuf/Immobilier%20Neuf"
]
base_url = "https://www.tayara.tn/ads/c/{}/?page={}"

def get_all_soup():
    soups = []
    for category in categories_path:
        for page in range(1, 50):  # Limiter à 5 pages pour l'exemple
            url = base_url.format(category, page)
            # print(url)
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            soups.append(soup)
    return soups

In [28]:
# permet de retourner tous les liens de chaque immobilier
# vs pouvez inspecter https://www.tayara.tn/item/66eae8a467b755ba922a2a58/Immobilier%20Neuf/Ariana/Ghazela/Appartement_en_S2_de_12880_m_A41_au_4me_tage/
def get_all_links(soup):
    property_links = []
    for article in soup.find_all('article', class_="mx-0"):
        link = article.find('a')['href']
        if '/item/' in link:
            full_url = "https://www.tayara.tn" + link
            property_links.append(full_url)
    return property_links

In [29]:
global_features = {} # contient toutes les caractéristiques possibles des biens immobiliers

def crawl_property_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    item_info = {}

    try:
        # Titre
        title_element = soup.find('h1', class_='text-gray-700 font-bold text-2xl font-arabic')
        if not title_element:
            # annonce gnrlmnt supprimée, ntaffiwha
            print(f"Titre non disponible pour l'URL: {url}, annonce ignorée.")
            return None
        title = title_element.text.strip()

        # Configuration des pièces
        pieces = re.search(r'S\+\d+', title)
        pieces_value = pieces.group() if pieces else None

        # Récupérer les parties du prix
        price_parts = soup.find_all('span', class_='mr-1')
        price = ''.join([part.text.strip() for part in price_parts if part.text.strip().isdigit()])

        # Localisation
        location_element = soup.find('div', class_='flex items-center space-x-2 mb-1')
        if location_element:
            location = location_element.text.strip().split(',')[0].strip()
        else:
            location = None

        # Ajouter les informations de base
        item_info['Annonce'] = title
        item_info['Prix'] = price
        item_info['Localisation'] = location
        item_info['Configuration des pièces'] = pieces_value

        # Ajouter dynamiquement les nouvelles clés à item_info
        criterias = soup.find_all('span', class_='text-gray-600/80 text-2xs md:text-xs lg:text-xs font-medium')
        values = soup.find_all('span', class_='text-gray-700/80 text-xs md:text-sm lg:text-sm font-semibold')

        # Associer chaque critère à sa valeur
        for criteria, value in zip(criterias, values):
            criterion_text = criteria.text.strip()
            value_text = value.text.strip()

            if criterion_text:
                # Ajouter la nouvelle clé à item_info
                item_info[criterion_text] = value_text
                # Ajouter cette clé dans global_features si elle n'existe pas encore
                if criterion_text not in global_features:
                    global_features[criterion_text] = 'False'

        # Vérifier si toutes les clés de global_features sont présentes dans item_info
        for feature in global_features:
            if feature not in item_info:
                item_info[feature] = 'False'
            

        return item_info

    except AttributeError as e:
        print(f"Erreur d'extraction à l'URL: {url} - {str(e)}")
        return None

In [30]:
property_links = []
for soup in get_all_soup():
    property_link = get_all_links(soup)
    property_links.extend(property_link)
print(property_links)
 # Exemple avec le lien https://www.tayara.tn/item/66eae8a467b755ba922a2a58/Immobilier%20Neuf/Ariana/Ghazela/Appartement_en_S2_de_12880_m_A41_au_4me_tage/
# data = crawl_property_page(property_links[0])
# print(data)

['https://www.tayara.tn/item/673c81c23c20a99fb553d9a1/Terrains%20et%20Fermes/Ariana/Raoued/Terrain_A_vendre_240M_/', 'https://www.tayara.tn/item/673c81be3c20a99fb553d99c/Locations%20de%20vacances/Sousse/Kantaoui/location_vacances/', 'https://www.tayara.tn/item/673c81960d8480a1e01226e8/Locations%20de%20vacances/Sousse/Khzema/par_nuite/', 'https://www.tayara.tn/item/673c81900d8480a1e01226e3/Appartements/Ariana/Ennasr/ENNASR_1_APPARTEMENT_S2_A_LOUER_/', 'https://www.tayara.tn/item/673c81733c20a99fb553d961/Locations%20de%20vacances/Sousse/Sousse/location_vacances_/', 'https://www.tayara.tn/item/673c816f0d8480a1e01226bf/Appartements/Ariana/El_Menzah_8/appartement_s2_Menzah_8/', 'https://www.tayara.tn/item/673c81513c20a99fb553d945/Maisons%20et%20Villas/Ariana/Les_Jardins_El_Menzah_2/REZDECHAUSSE_DE_VILLA_S4_AUX_JARDINS_DEL_MENZAH_2/', 'https://www.tayara.tn/item/673c81030d8480a1e0122658/Appartements/Sousse/Sahloul/appartement_de_type_S2_situ_sahloul/', 'https://www.tayara.tn/item/673c81483c2

In [31]:
print(property_links.__len__())

1667


In [34]:
def save_to_csv(data, filename="immobiliers.csv"):
    if not data:
        print("Aucune donnée à sauvegarder.")
        return

    # Créer un ensemble pour les fieldnames
    fieldnames = set()

    # Passer par toutes les annonces et ajouter dynamiquement les champs
    for row in data:
        if row:  # Vérifier que la ligne n'est pas vide
            fieldnames.update(row.keys())  # Ajouter toutes les clés des données à fieldnames

    # Convertir l'ensemble en une liste pour avoir une structure ordonnée
    fieldnames = list(fieldnames)

    try:
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for row in data:
                if row:  # Vérifier que la ligne n'est pas vide
                    writer.writerow(row)

    except PermissionError:
        print(f"Erreur: permission refusée pour écrire dans le fichier {filename}.")
    except Exception as e:
        print(f"Erreur lors de l'enregistrement des données : {e}")

In [35]:
data = []
for link in property_links:
    try:
        data.append(crawl_property_page(link))
    except requests.exceptions.ConnectionError as e:
        print(f"Connection error: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Other error: {e}")
    except RemoteDisconnected as e:
        print(f"Remote disconnected: {e}")
save_to_csv(data, "immobiliers.csv")

Titre non disponible pour l'URL: https://www.tayara.tn/item/673c80f70d8480a1e0122643/Appartements/Tunis/L_Aouina/a_vendre_un_magnifique_appartement_s3_a_laouina_cit_wahat_/, annonce ignorée.
Titre non disponible pour l'URL: https://www.tayara.tn/item/673c80db3c20a99fb553d8b7/Appartements/Tunis/Ain_Zaghouen/a_vendre_un_appartement_au_rdc_a_ain_zaghouan_avec_place_parking_soussol_/, annonce ignorée.
Titre non disponible pour l'URL: https://www.tayara.tn/item/673c80b90d8480a1e012260e/Appartements/Tunis/L_Aouina/A_louer_S2/, annonce ignorée.
Titre non disponible pour l'URL: https://www.tayara.tn/item/673c7c9f0d8480a1e0122276/Maisons%20et%20Villas/Ariana/La_Soukra/_a_vendre_une_villa_S3_avec_piscine_a_la_soukra_/, annonce ignorée.
Titre non disponible pour l'URL: https://www.tayara.tn/item/673c7c7d3c20a99fb553d4d6/Appartements/Tunis/L_Aouina/A_vendre_des_appartements_direct_promoteur_haut_standing_a_la_nouvelle_soukra/, annonce ignorée.
Titre non disponible pour l'URL: https://www.tayara.tn